In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import spacy
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import RegexpTokenizer, WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from string import punctuation
import collections
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import en_core_web_sm
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score

ModuleNotFoundError: No module named 'en_core_web_sm'

In [119]:
import pandas as pd
import pyodbc 

In [173]:
cursor = cnxn.cursor()

In [184]:
rows = cursor.execute("SELECT * from dbo.Users1").fetchmany(4)

In [185]:
df = pd.DataFrame((tuple(t) for t in rows), columns=['id',
 'author',
 'created_at',
 'location',
 'description',
 'verified',
 'followers',
 'following',
 'favourites_count',
 'statuses_count',
 'lang',
 'tweets',
 'following_json',
 'followers_json']) 


In [186]:
df.head()

,id,author,created_at,location,description,verified,followers,following,favourites_count,statuses_count,lang,tweets,following_json,followers_json
0,2284857094,themoonisironic,2014-01-16 01:10:33,‘97 // they/them // ????????,"where there is hope, there are trials // art: ...",False,101,516,121166,36377,None,"[{""id"": 1356030533780824069, ""created_at"": ""20...","[{""id"": 1387079251476815875, ""author"": ""rikuno...","[{""id"": 1384297303129825284, ""author"": ""bts123..."
1,834324620,bbelita23,2012-09-19 23:21:48,"spooky town, PR",just put it out into the universe,False,670,443,33428,92917,None,"[{""id"": 1356026926134140928, ""created_at"": ""20...","[{""id"": 2513536418, ""author"": ""Kvn_Astacio"", ""...","[{""id"": 1289669319526420482, ""author"": ""keviin..."
2,175521394,Stormfocus18,2010-08-06 21:03:08,Puerto Rico_Island,"Loving #MewGulf and #KristSingto, amante de la...",False,199,187,6497,38945,None,[],"[{""id"": 1387677517822054400, ""author"": ""tinest...","[{""id"": 1387677517822054400, ""author"": ""tinest..."
3,1685491040,CoraimaINegron,2013-08-20 10:48:59,Puerto Rico,"“El karma te lo devolverá todo, excepto a la m...",False,64,203,1336,1565,None,"[{""id"": 1283388114661244934, ""created_at"": ""20...","[{""id"": 299932350, ""author"": ""DMcIntyreWWE"", ""...","[{""id"": 713749428935462916, ""author"": ""DavidRo..."


In [187]:
import json

for x in df.tweets: 
    y = json.loads(x)
    
tweets = pd.DataFrame(y) 
tweets.head()


,id,created_at,text,coordinates,geo,user,retweet_count,favorite_count
0,1283388114661244934,2020-07-15T13:09:05,Nada más cierto! 🤦🏻‍♀️🤦🏻‍♀️🤦🏻‍♀️ https://t.co/...,None,None,CoraimaINegron,0,0
1,1282263167511470082,2020-07-12T10:38:57,Llamen a mi jefe diganle que la lluvia no me d...,None,None,CoraimaINegron,0,0
2,1280867816544047107,2020-07-08T14:14:19,RT @SaraJarque: https://t.co/olOHb7CC9R,None,None,CoraimaINegron,403,0
3,1280867775209234433,2020-07-08T14:14:09,"RT @SaraJarque: 1, 2, 3 Pescao! Pescao! https:...",None,None,CoraimaINegron,205,0
4,1280439524485718022,2020-07-07T09:52:26,@kefvelazquez Así está bien o todavía no estas...,None,None,CoraimaINegron,0,1


In [98]:
# remove the hashtags, mentions and unwanted characters.
def clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0–9]+)|([⁰-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", “”, elem)) 
    return df
tweets_bowl = clean_text(tweets, ‘tweets’)
tweets_bowl.head()```

SyntaxError: invalid character '“' (U+201C) (<ipython-input-98-b407aec837de>, line 4)

In [170]:
tweets.head()

,id,created_at,text,coordinates,geo,user,retweet_count,favorite_count
0,1356040835469635584,2021-02-01T00:45:03,RT @pipitostgo: Acho si fuera Yulin la que est...,None,None,Gatacielo,129,0
1,1355630751933493254,2021-01-30T21:35:32,RT @OComejen: -ROMPIENDO-\nAparece el fantasma...,None,None,Gatacielo,5,0
2,1355630670706577408,2021-01-30T21:35:12,RT @josecrates: El panorama mundial es de no a...,None,None,Gatacielo,5,0
3,1355626611085893642,2021-01-30T21:19:05,RT @LISI_13: Y el gran problema son los confor...,None,None,Gatacielo,3,0
4,1355624457117827072,2021-01-30T21:10:31,RT @elsbtmar: Allá los que les encanta que les...,None,None,Gatacielo,30,0


In [198]:
import preprocessor as p
p.set_options(p.OPT.URL,p.OPT.MENTION,p.OPT.HASHTAG,p.OPT.RESERVED,p.OPT.SMILEY,p.OPT.NUMBER)
p.clean('Preprocessor í is #awesome 👍 https://github.com/s/preprocessor')

'Preprocessor í is 👍'

In [199]:
tuits = []
for tweet in tweets.text:
    tuits.append(p.clean(tweet))

In [215]:
import nltk 


'''Natural Language Processing libraries'''
import nltk 
import gensim
import regex as re
import spacy
from spacy.lang.en import English
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import gensim.downloader as api
import re, string, unicodedata
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize
from gensim.models import FastText
from gensim.models import Word2Vec



class Document:
    """ Retrieve the narratives from the DataFrame and respectively
        store and pre-process it. 
        
        :param df: DataFrame including the reports and the predictor variable. 
        
        
        :ivar data: Stores the DataFrame.
        :ivar text: Stores the narratives as string.
        :ivar corpus: Stores the pre-processed text.
    """
    
    
    def __init__(self, df):
        self.data = df
        self.text = df["text"].astype(str)
        self.textPreProcessing()
        
        
    def remove_non_ascii(self, words):
        """Remove non-ASCII characters from list of tokenized words
        
        :param words:  List of words to be transformed when removing non_ascii characters.
        
        :return new_words: List of words after the transformation of removed non_ascii characters.
        
        """
        new_words = []
        for word in words:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
        return new_words


    def remove_punctuation(self, words):
        """Remove punctuation from list of tokenized words
        
        :param words:  List of words that will get remove their punctuations, if any. 
        
        :return new_words: List of transformed words.
        
        
        """
        new_words = []
        for word in words:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
        return new_words


    def stem_words(self, words):
        """Stem words in list of tokenized words
        
        :param words:  List of words to be processed. 
        
        :return new_words: List of the received words respective stems.
        
        
        """
        
        stemmer = LancasterStemmer()
        stems = []
        for word in words:
            stem = stemmer.stem(word)
            stems.append(stem)
        return stems
    
    def lemmatize_verbs(self, words):
        """Lemmatize verbs in list of tokenized words
        
        :param words:  List of words to be processed. 
        
        :return new_words: List of the received words respective lemmas.
        
        """
        
        lemmatizer = WordNetLemmatizer()
        lemmas = []
        for word in words:
            lemma = lemmatizer.lemmatize(word, pos='v')
            lemmas.append(lemma)
        return lemmas
    
    
    
    def remove_stopwords(self, words):
        """Remove common words that have no meaning or importance in the sentence.

        :param words:  List of words to be processed and get stop words removed.. 

        :return new_words: List of words with the stop words already removed."""
            
        
        stop_words = set(stopwords.words('spanish')) 
        
        for word in stop_words:
            if word in words:
                words.remove(word)
                
        return words


    
    def normalize(self, words):
        words = self.remove_non_ascii(words)
        words = self.remove_stopwords(words)
        words = self.remove_punctuation(words)
        words = self.lemmatize_verbs(words)
        return words
    
    
    def textPreProcessing(self):
        """Pre-process the text, normalize and clean it.
        The function stores the cleaned text in the self.data
        attribute. """

        clean_text = []

        for narrative in self.text:
            
            sentence = word_tokenize(narrative)
            sentence = self.normalize(sentence)
                
                
            clean_text.append(sentence)
            
            
                    
        print(len(self.text), len(clean_text))
        self.data["clean_text"] = clean_text

In [200]:
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')


texto = pd.DataFrame(tuits, columns=["text"])

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/bryanortiz/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/bryanortiz/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/bryanortiz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [216]:
nltk.download('stopwords')
nltk.download('wordnet')
documento = Document(texto)


118 118


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bryanortiz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bryanortiz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [217]:
documento.data["clean_text"]

0                                    [Nada, mas, cierto]
1      [Llamen, jefe, diganle, lluvia, deja, ir, trab...
2                                                     []
3                                       [Pescao, Pescao]
4      [Asi, bien, todavia, tan, sorprendido, El, gol...
                             ...                        
113    [El, ajusta, magnitud, limite, producir, tsunami]
114    [Van, seguir, ocurriendo, eventos, fuertes, Ha...
115                             [Casa, colapsada, Yauco]
116    [Las, replicas, podrian, ser, magnitud, mayor,...
117    [PRELIMINAR, 0106, 5803, No, Aviso, Advertenci...
Name: clean_text, Length: 118, dtype: object

In [238]:
model = Word2Vec(documento.data["clean_text"], min_count=2, workers=20, window=2,  alpha=0.02, hs=1)



In [240]:
my_dict = dict({})
for idx, key in enumerate(model.wv.key_to_index):
    my_dict[key] = model.wv[key]
    # Or my_dict[key] = model.wv.get_vector(key)
    # Or my_dict[key] = model.wv.word_vec(key, use_norm=False)

In [250]:
vectores = []
for a in documento.data.clean_text:
    t  = []
    for word in a:
        try:
            t.append(my_dict[word])
        except:
            continue
    
    vectores.append(t)
        
documento.data["vectorized_text"] = vectores
documento.data.head()

,text,clean_text,vectorized_text
0,Nada más cierto! 🤦🏻‍♀️🤦🏻‍♀️🤦🏻‍♀️,"[Nada, mas, cierto]","[[0.009654882, 0.007363939, 0.0012740177, -0.0..."
1,Llamen a mi jefe diganle que la lluvia no me d...,"[Llamen, jefe, diganle, lluvia, deja, ir, trab...","[[0.009929657, 0.0024249798, 0.0051007434, -0...."
2,:,[],[]
3,": , , Pescao! Pescao!","[Pescao, Pescao]","[[0.0023572692, 0.00421269, 0.0019974997, 0.00..."
4,Así está bien o todavía no estas tan sorprendi...,"[Asi, bien, todavia, tan, sorprendido, El, gol...","[[0.004190841, 0.002390885, -0.0003632684, -0...."


In [248]:
my_dict["Van"]

KeyError: 'Van'

In [234]:
word2vec_model.wv.key_to_index

{'de': 0,
 'No': 1,
 'a': 2,
 'Aviso': 3,
 'Bayamon': 4,
 'Nuevo': 5,
 'Barrio': 6,
 'El': 7,
 'la': 8,
 'magnitud': 9,
 'Ml': 10,
 'Sentido': 11,
 'Km': 12,
 'Prof': 13,
 'Mag': 14,
 'PRELIMINAR': 15,
 'Max': 16,
 'Puerto': 17,
 'Rico': 18,
 'Int': 19,
 'temblor': 20,
 'PR': 21,
 'enero2020': 22,
 'PM': 23,
 'que': 24,
 'mas': 25,
 'tsunami': 26,
 'be': 27,
 'sabado': 28,
 'luz': 29,
 'servicio': 30,
 'hoy': 31,
 'La': 32,
 'Gracias': 33,
 'los': 34,
 'Wanda': 35,
 'Hoy': 36,
 'mayor': 37,
 'mejor': 38,
 'ser': 39,
 'el': 40,
 'vez': 41,
 'Que': 42,
 'Mayaguez': 43,
 'gente': 44,
 'fuerte': 45,
 'ver': 46,
 'si': 47,
 'l': 48,
 'Mas': 49,
 'cada': 50,
 'ir': 51,
 'chicharra': 52,
 'con': 53,
 'preliminar': 54,
 'Islas': 55,
 'Advertencia': 56,
 'Vigilancia': 57,
 'no': 58,
 'Los': 59,
 'ACTUALIZADA': 60,
 'temblores': 61,
 'pandemia': 62,
 'Lo': 63,
 'Enero': 64,
 'tan': 65,
 'espero': 66,
 'isla': 67,
 'Hola': 68,
 'cierto': 69,
 'Pescao': 70,
 'bien': 71,
 'vi': 72,
 'NO': 73,
 'ent

In [118]:
# remove the hashtags, mentions and unwanted characters.
def clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0–9]+)|([⁰-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "",   elem)) 
    return df
tweets_bowl = clean_text(tuits, 'text')
tweets_bowl.head()

TypeError: list indices must be integers or slices, not str